In [37]:
import pandas as pd
import glob
import datetime

In [14]:
used_columns_files = '数据逻辑梳理.xlsx'
item_def = pd.read_excel(used_columns_files)

In [47]:
        cr_src = pd.DataFrame()
        for year in [2018, 2019, 2020, 2021, 2022, 2023]:
            cr_src_yr = pd.read_excel("data/PROCUREMENT_China_23 Nov.xlsx", sheet_name=f"{year}")
            cr_src_yr['year'] = year
            cr_src = pd.concat([cr_src, cr_src_yr])

In [48]:
cr_src['ID'] = cr_src['ID'].astype(str).str.split('.').apply(lambda x: "-".join([str(list(x)[0]),str(list(x)[1])[:6]]))

In [50]:
turnover_items = item_def['DB_TO_FTE_LIST'].tolist()
turnover_files = glob.glob('data/Turnover*')
turnover_data = pd.DataFrame()
for i,file in enumerate(turnover_files):
    print(file)
    data = pd.read_excel(file, sheet_name='DB_TO_FTE_LIST')
    data = data[turnover_items]
    data['file_id'] = i
    turnover_data = pd.concat([turnover_data, data])

data\Turnover Report_Procurement_201812_Final.xlsx
data\Turnover Report_Procurement_201912_Final.xlsx
data\Turnover Report_Procurement_202012-final-exclude DGO.xlsx
data\Turnover Report_Procurement_202112.xlsx
data\Turnover Report_Procurement_202212.xlsx
data\Turnover Report_Procurement_202310 APAC.xlsx


In [51]:
turnover_data['ID'] = turnover_data['ID'].astype(str).str.split('.').apply(lambda x: "-".join([str(list(x)[0]),str(list(x)[1])[:6]]))

In [3]:
movement = pd.read_excel('data/PROCUREMENT_China_23 Nov.xlsx')

In [25]:
movement['ID'] = movement.apply(lambda x: "-".join([str(x['ID1']),str(x['ID2'])]), axis=1)

In [27]:
movement['ID']

0      34508121-9572471
1      34510267-8201798
2      52650622-1929166
3      52658359-1399952
4      52691378-3306411
             ...       
606    52724869-7338338
607    52724880-9998634
608    63722125-5408532
609    52724876-3056847
610    52724886-6328774
Name: ID, Length: 611, dtype: object

In [10]:
turnover_data = pd.read_excel('step_data/turnover_data.xlsx')

C:\Users\28079083\AppData\Local\Temp\ipykernel_12836\985957999.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  turnover_data = pd.read_excel('step_data/turnover_data.xlsx')


In [13]:
turnover_data['ID'].astype(str).str.split('.')

0      [52665887, 2357789]
1      [52666391, 0129865]
2      [52691278, 7506866]
3      [52691784, 1639811]
4      [52704339, 0282876]
              ...         
299                 [1, 0]
300                 [1, 0]
301                 [1, 0]
302                 [1, 0]
303                 [1, 0]
Name: ID, Length: 304, dtype: object

In [40]:
def get_salary_change(userid, current_date):
    salary_features = {}
    salary_data = cr_src[(cr_src['ID'] == userid) & (current_date > cr_src['Hire Date'])]
    if len(salary_data) > 0:
        recent_cr = salary_data.sort_values(by='Hire Date', ascending=False)['Compa Ratio'].tolist()[0]  # 最近薪资
        recent_cr_change_time = salary_data['Hire Date'].max()

        try:
            last_cr = salary_data.sort_values(by='Hire Date', ascending=False)['Compa Ratio'].tolist()[1]  # 再之前的薪资
            recent_cr_diff = recent_cr - last_cr
        except:
            last_cr = None
            recent_cr_diff = None

        salary_features['recent_cr'] = recent_cr
        salary_features['days_since_recent_cr_change'] = (pd.to_datetime(current_date) - recent_cr_change_time).days
        salary_features['cr_change_times'] = salary_data.shape[0]
        salary_features['recent_cr_diff'] = recent_cr_diff
    return salary_features


def get_salary_features(fte):
    # 获取salary的标签
    final_data = []
    for i, item in fte.iterrows():
        exist_items = dict(item)
        userid = exist_items['ID']
        end_date = exist_items['current_year']
        salary_features = get_salary_change(userid, end_date)
        exist_items.update(salary_features)
        final_data.append(exist_items)
    final_data = pd.DataFrame(final_data)
    final_data['cr_change_vs_term'] = final_data['cr_change_times'] / final_data['on_duty_days'] * 365
    return final_data

In [52]:
turnover_data['used_region_flag'] = turnover_data.apply(lambda x: define_used_region(x), axis=1)
turnover_data = turnover_data.query('(used_region_flag > 0) & (LeaveType == "Termination-Voluntary")')
# turnover_data = turnover_data.query('ID != 52710485.7444984')     # onboardDate数据错误
turnover_data['OnboardDate'] = pd.to_datetime(turnover_data['OnboardDate'].astype(str))
turnover_data['LastWorkingdate'] = pd.to_datetime(turnover_data['LastWorkingdate'])
turnover_data['current_year'] = turnover_data['LastWorkingdate'] - datetime.timedelta(days=31*6)
turnover_data['on_duty_days'] = (turnover_data['current_year'] - turnover_data['OnboardDate']).dt.days    # 截止到半年前已经就职了多少天
turnover_data['current_year'] = turnover_data['current_year'].dt.strftime("%Y-%m-%d")
turnover_data2 = get_salary_features(turnover_data)

In [53]:
turnover_data2.to_excel('data/check/check_cr.xlsx', index=False)

In [46]:
cr_src.to_excel('data/check/cr_src.xlsx',index=False)